# SVM Classification
### 타이타닉 생존자 분류

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
# Seaborn에서 제공하는 titanic 데이터셋 가져오기
df = sns.load_dataset('titanic')
print(df)                                # [891 rows x 15 columns]

     survived  pclass     sex   age  sibsp  parch     fare embarked   class  \
0           0       3    male  22.0      1      0   7.2500        S   Third   
1           1       1  female  38.0      1      0  71.2833        C   First   
2           1       3  female  26.0      0      0   7.9250        S   Third   
3           1       1  female  35.0      1      0  53.1000        S   First   
4           0       3    male  35.0      0      0   8.0500        S   Third   
..        ...     ...     ...   ...    ...    ...      ...      ...     ...   
886         0       2    male  27.0      0      0  13.0000        S  Second   
887         1       1  female  19.0      0      0  30.0000        S   First   
888         0       3  female   NaN      1      2  23.4500        S   Third   
889         1       1    male  26.0      0      0  30.0000        C   First   
890         0       3    male  32.0      0      0   7.7500        Q   Third   

       who  adult_male deck  embark_town alive  alo

In [3]:
#  IPython 디스플레이 설정 - 출력할 열의 개수 한도 늘리기
pd.set_option('display.max_columns', 15)
print(df.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  


In [4]:
# NaN값이 많은 deck(배의 갑판)열을 삭제 : deck 열은 유효한 값이 203개
# embarked(승선한)와 내용이 겹치는 embark_town(승선 도시) 열을 삭제
# 전체 15개의 열에서  deck, embark_town 2개의 열이 삭제되어서 13개의 열이름만 출력
rdf = df.drop(['deck', 'embark_town'], axis=1)  
print(rdf.columns.values)

['survived' 'pclass' 'sex' 'age' 'sibsp' 'parch' 'fare' 'embarked' 'class'
 'who' 'adult_male' 'alive' 'alone']


In [5]:
# 승객의 나이를 나타내는 age 열에 누락 데이터가 177개 포함되어 있다.
# 누락 데이터를 평균 나이로 치환하는 방법도 가능하지만, 누락 데이터가 있는 행을 모두 삭제한다.
# 즉, 177명의 승객 데이터를 포기하고 나이 데이터가 있는 714명의 승객만을 분석 대상으로 한다.
# age 열에 나이 데이터가 없는 모든 행을 삭제 - age 열(891개 중 177개의 NaN 값)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
print(len(rdf))                           # 714  (891개 중 177개 데이터 삭제)

714


In [6]:
# embarked 열에는 승객들이 타이타닉호에 탑승한 도시명의 첫 글자가 들어있다.
# embarked 열에는 누락 데이터(NaN)가 2개에 있는데, 누락 데이터를 가장 많은 도시명(S)으로 치환한다.
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
# value_counts()함수와 idxmax()함수를 사용하여 승객이 가장 많이 탑승한 도시명의 첫 글자는 S
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
print(most_freq)                          # S : Southampton

S


In [7]:
# embarked 열의 최빈값(top)을 확인하면 S 로 출력됨
print(rdf.describe(include='all'))

          survived      pclass   sex         age       sibsp       parch  \
count   714.000000  714.000000   714  714.000000  714.000000  714.000000   
unique         NaN         NaN     2         NaN         NaN         NaN   
top            NaN         NaN  male         NaN         NaN         NaN   
freq           NaN         NaN   453         NaN         NaN         NaN   
mean      0.406162    2.236695   NaN   29.699118    0.512605    0.431373   
std       0.491460    0.838250   NaN   14.526497    0.929783    0.853289   
min       0.000000    1.000000   NaN    0.420000    0.000000    0.000000   
25%       0.000000    1.000000   NaN   20.125000    0.000000    0.000000   
50%       0.000000    2.000000   NaN   28.000000    0.000000    0.000000   
75%       1.000000    3.000000   NaN   38.000000    1.000000    1.000000   
max       1.000000    3.000000   NaN   80.000000    5.000000    6.000000   

              fare embarked  class  who adult_male alive alone  
count   714.000000    

In [8]:
# embarked 열에 fillna() 함수를 사용하여 누락 데이터(NaN)를 S로 치환한다.
rdf['embarked'].fillna(most_freq, inplace=True)

In [9]:
# 분석에 사용할 열(속성)을 선택
ndf = rdf[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'embarked']]

In [10]:
# SVM 모델을 적용하기 위해 sex열과 embarked열의 범주형 데이터를 숫자형으로 변환한다.
# 이 과정을 더미 변수를 만든다고 하고, 원핫인코딩(one-hot-encoding)이라고 부른다.
# 원핫인코딩 - 범주형 데이터를 모델이 인식할 수 있도록 숫자형으로 변환 하는것
# sex 열은 male과 female값을 열 이름으로 갖는 2개의 더미 변수 열이 생성된다.
# concat()함수로 생성된 더미 변수를 기존 데이터프레임에 연결한다.
onehot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, onehot_sex], axis=1)

In [11]:
# embarked 열은 3개의 더미 변수 열이 만들어지는데, prefix='town' 옵션을 사용하여
# 열 이름에 접두어 town을 붙인다. ( town_C, town_Q, town_S)
onehot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, onehot_embarked], axis=1)

In [12]:
# 기존 sex열과 embarked열 삭제
ndf.drop(['sex', 'embarked'], axis=1, inplace=True)

In [13]:
# 변수 정의
x=ndf[['pclass', 'age', 'sibsp', 'parch', 'female', 'male', 'town_C', 'town_Q', 'town_S']]   #독립 변수 x
y=ndf['survived']                                                                            #종속 변수 y

In [14]:
# 독립 변수 데이터를 정규화(normalization)
# 독립 변수 열들이 갖는 데이터의 상대적 크기 차이를 없애기 위하여 정규화를 한다.
from sklearn import preprocessing
x = preprocessing.StandardScaler().fit(x).transform(x)

In [15]:
# train data 와 test data로 구분(7:3 비율)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [16]:
print('train data 개수: ', x_train.shape)          # train data 개수:  (499, 9)
print('test data 개수: ', x_test.shape)            # test data 개수:  (215, 9)

train data 개수:  (499, 9)
test data 개수:  (215, 9)


In [17]:
# sklearn 라이브러리에서 SVM 분류 모델 가져오기
from sklearn import svm

In [18]:
# 모델 객체 생성 (kernel='rbf' 적용)
svm_model = svm.SVC(kernel='rbf')

In [19]:
# train data를 가지고 모델 학습
svm_model.fit(x_train, y_train)

SVC()

In [20]:
# test data를 가지고 y_hat을 예측 (분류) 
y_hat = svm_model.predict(x_test)                # 예측값 구하기

In [21]:
# 10개의 예측값(y_hat)과 실제값(y_test) 비교
print(y_hat[0:10])                               # [0 0 1 0 0 0 1 0 0 0]
print(y_test.values[0:10])                       # [0 0 1 0 0 1 1 1 0 0]

[0 0 1 0 0 0 1 0 0 0]
[0 0 1 0 0 1 1 1 0 0]


In [22]:
# 모델 성능 평가 - Confusion Matrix(혼동 행렬) 계산
from sklearn import metrics 
svm_matrix = metrics.confusion_matrix(y_test, y_hat)  
print(svm_matrix)

[[120   5]
 [ 35  55]]


In [23]:
# 모델 성능 평가 - 평가지표 계산
svm_report = metrics.classification_report(y_test, y_hat)            
print(svm_report)

# f1지표(f1-score)는 모델의 예측력을 종합적으로 평가하는 지표이다.
# f1-score 지표를 보면 사망자(0) 예측의 정확도가 0.86이고, 생존자(1) 예측의 
# 정확도는 0.73으로 예측 능력에 차이가 있다.
# 전반적으로 KNN모델의 예측 능력과 큰 차이가 없다.

              precision    recall  f1-score   support

           0       0.77      0.96      0.86       125
           1       0.92      0.61      0.73        90

    accuracy                           0.81       215
   macro avg       0.85      0.79      0.80       215
weighted avg       0.83      0.81      0.81       215

